In [65]:
import os
import pandas as pd

import py2neo
from py2neo import Graph,Node,Relationship

import IPython

print ('py2neo version', py2neo.__version__)

py2neo version 2021.2.3


In [66]:
db_url = 'neo4j@bolt://neo4j@localhost:7687'
db_pass = 'tweets'

graph = Graph(db_url, password = db_pass)

In [67]:
#consulta de prueba para ver que todo funcione

nro_nodos_user="MATCH(u:User) RETURN Count(u)"
nro_nodos_tweet="MATCH(t:Tweet) RETURN Count(t)"

#evaluamos las consultas
result_users=graph.evaluate(nro_nodos_user)
result_tweets=graph.evaluate(nro_nodos_tweet)

#imprimo resultados
print(f'En la BD hay {result_users} nodos User y {result_tweets} nodos Tweet')

En la BD hay 55860 nodos User y 405369 nodos Tweet


In [41]:
# query for dropping a projection from the catalog
query = (
    "CALL gds.graph.drop('tweets') "
)

result = graph.run(query).to_data_frame()

result

,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
0,tweets,neo4j,,-1,474112,1098318,{'relationshipProjection': {'QUOTE': {'orienta...,0.000005,2022-06-13T18:32:47.873189000-03:00,2022-06-13T18:32:50.980297999-03:00,"{'relationships': {'QUOTE': {}, 'TAG': {}, 'ME..."


In [48]:
# Create a projection of the graph and store it on the catalog
# We use the user projection described in https://towardsdatascience.com/using-neo4j-graph-database-to-analyze-twitter-data-6e3d38042af1
user_projecion = 'MATCH (u:User) return id(u) as id'
relationships_projection = (
    "MATCH (u1:User)-[:TWEETED]->(t1:Tweet)<-[:QUOTE|REPLY]-(t2:Tweet)<-[:TWEETED]-(u2:User) "
    "RETURN id(u2) as source, id(u1) as target "
    "UNION "
    "MATCH (u1:User)<-[:MENTIONS]-(t1:Tweet)<-[:QUOTE|REPLY]-(t2:Tweet)<-[:TWEETED]-(u2:User) "
    "RETURN id(u2) as source, id(u1) as target "
    "UNION "
    "MATCH (u1:USER)<-[:MENTIONS]-(t1:Tweet)<-[:TWEETED]-(u2:User) "
    "RETURN id(u2) as source, id(u1) as target "
)

nodos_tweet = (
    f"CALL gds.graph.project.cypher('tweets', '{user_projecion}', '{relationships_projection}') "
)

#evaluamos las consultas
tweets=graph.run(nodos_tweet).to_data_frame() 

tweets

,nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
0,MATCH (u:User) return id(u) as id,MATCH (u1:User)-[:TWEETED]->(t1:Tweet)<-[:QUOT...,tweets,55860,187877,3515


In [89]:
# Retrieve all graphs on the catalog to see if it was stored correctly

query = (
    "CALL gds.graph.list() "
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,degreeDistribution,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
0,"{'p99': 57, 'min': 0, 'max': 961, 'mean': 3.36...",tweets,neo4j,8786 KiB,8997271,55860,187877,{'relationshipQuery': 'MATCH (u1:User)-[:TWEET...,0.000060,2022-06-21T19:13:03.148202000-03:00,2022-06-21T19:13:06.701191000-03:00,"{'relationships': {'__ALL__': {}}, 'nodes': {'..."
1,"{'p99': 22, 'min': 0, 'max': 3502, 'mean': 2.3...",complete,neo4j,63 MiB,66118975,474112,1098318,{'relationshipProjection': {'QUOTE': {'orienta...,0.000005,2022-06-30T07:28:18.507761000-03:00,2022-06-30T07:28:21.439054000-03:00,"{'relationships': {'QUOTE': {}, 'TAG': {}, 'ME..."


In [90]:
# Execute the PageRank centrality algorithm
user_info = (
    "MATCH (u:User)-[:TWEETED]->(t:Tweet) "
    "WHERE u.username = node.username "
    "WITH u, score, count(t) as may_tweets "
    "RETURN u.name as Name, u.username as Username, u.followers as Followers, u.tweet_count as TotalTweets, score as Score, may_tweets as MayTweets "
    "ORDER BY Score DESC, Name ASC "
    "LIMIT 20"
)

query = (
    "CALL gds.pageRank.stream('complete') "
    "YIELD nodeId, score "
    "WITH gds.util.asNode(nodeId) AS node, score "
    f"{user_info}"
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,Name,Username,Followers,TotalTweets,Score,MayTweets
0,Isabel_,Isabel66991411,5478,78777,532.363366,3502
1,Nacional,Nacional,341359,29775,258.826929,154
2,☪️🅰️®️🛴🌎💰,boniatero75,2129,42533,244.961132,1943
3,PEÑAROL,OficialCAP,434933,43753,212.705624,142
4,Luis Lacalle Pou,LuisLacallePou,416367,27270,212.469123,26
5,JACK_TORRANCE⬆️,LaMoscaFliesu2,638,13809,196.414486,2151
6,LMP,Lelengo86,513,5257,142.805804,1586
7,💫💫Estela❤️😷😷🏆🖤💛,EstelaF68436226,3484,84862,139.781317,1404
8,Maria Gagliardi..@2311,mag_2311,6847,105786,130.684302,875
9,gaby 💖💙❤️,sgabyo,2084,28238,130.111393,1820


In [91]:
# Execute the ArticleRank centrality algorithm

query = (
    "CALL gds.articleRank.stream('complete') "
    "YIELD nodeId, score "
    "WITH gds.util.asNode(nodeId) AS node, score "
    f"{user_info}"
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,Name,Username,Followers,TotalTweets,Score,MayTweets
0,Isabel_,Isabel66991411,5478,78777,137.856228,3502
1,Nacional,Nacional,341359,29775,74.258115,154
2,Luis Lacalle Pou,LuisLacallePou,416367,27270,68.426836,26
3,☪️🅰️®️🛴🌎💰,boniatero75,2129,42533,66.749522,1943
4,PEÑAROL,OficialCAP,434933,43753,65.690199,142
5,JACK_TORRANCE⬆️,LaMoscaFliesu2,638,13809,59.331938,2151
6,💫💫Estela❤️😷😷🏆🖤💛,EstelaF68436226,3484,84862,43.618648,1404
7,EL PAÍS,elpaisuy,792004,417586,41.687620,398
8,ANALI BENTENCOURT,Analiros,1373,23096,40.171233,1624
9,Graciela Bianchi,gbianchi404,74276,60786,39.878694,137


In [93]:
# Execute the Betweenness centrality algorithm

query = (
    "CALL gds.betweenness.stream('complete') "
    "YIELD nodeId, score "
    "WITH gds.util.asNode(nodeId) AS node, score "
    f"{user_info}"
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,Name,Username,Followers,TotalTweets,Score,MayTweets
0,Isabel_,Isabel66991411,5478,78777,7.304500e+09,3502
1,☪️🅰️®️🛴🌎💰,boniatero75,2129,42533,4.205308e+09,1943
2,gaby 💖💙❤️,sgabyo,2084,28238,3.648774e+09,1820
3,JACK_TORRANCE⬆️,LaMoscaFliesu2,638,13809,3.090501e+09,2151
4,LMP,Lelengo86,513,5257,2.456070e+09,1586
5,ANALI BENTENCOURT,Analiros,1373,23096,2.403511e+09,1624
6,Jorge Andrés,JorgeAndresBusi,2554,94890,2.199247e+09,1017
7,𝕴𝖓𝖘𝖔𝖕𝖔𝖗𝖙𝖆𝖇𝖑𝖊𝖒𝖊𝖓𝖙𝖊 𝕭𝖔𝖑𝖘𝖔,KiwiNacional,1355,53954,1.846585e+09,941
8,💫💫Estela❤️😷😷🏆🖤💛,EstelaF68436226,3484,84862,1.805920e+09,1404
9,Antonella Gordillo 🇺🇾🥕,AntoGordillo,5616,14396,1.718331e+09,785


In [92]:
# Execute the Degree centrality algorithm

query = (
    "CALL gds.degree.stream('complete') "
    "YIELD nodeId, score "
    "WITH gds.util.asNode(nodeId) AS node, score "
    f"{user_info}"
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,Name,Username,Followers,TotalTweets,Score,MayTweets
0,Isabel_,Isabel66991411,5478,78777,3502.0,3502
1,JACK_TORRANCE⬆️,LaMoscaFliesu2,638,13809,2151.0,2151
2,☪️🅰️®️🛴🌎💰,boniatero75,2129,42533,1943.0,1943
3,gaby 💖💙❤️,sgabyo,2084,28238,1820.0,1820
4,𝚂iL ❤️‍🔥 #36303,madrechocolates,977,77480,1646.0,1646
5,Jenny Parada Martino,MartinoParada,1137,62402,1625.0,1625
6,ANALI BENTENCOURT,Analiros,1373,23096,1624.0,1624
7,LMP,Lelengo86,513,5257,1586.0,1586
8,Cada dirección de Montevideo,everylotMVD,251,14198,1488.0,1488
9,💫💫Estela❤️😷😷🏆🖤💛,EstelaF68436226,3484,84862,1404.0,1404


In [86]:
nodos_tweet = (
    f"CALL gds.graph.project('complete', ['User', 'Tweet', 'Hashtag'], ['TWEETED', 'MENTIONS', 'QUOTE', 'REPLY', 'TAG']) "
)

#evaluamos las consultas
tweets=graph.run(nodos_tweet).to_data_frame() 

tweets

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Hashtag': {'label': 'Hashtag', 'properties':...","{'QUOTE': {'orientation': 'NATURAL', 'aggregat...",complete,474112,1098318,2770
